# Write Core Data into TEI files


## Packages & Functions

In [1]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.dom import minidom
import pandas as pd
import os

import re

In [2]:
### Function that remove "arrays" from the author fields

def removeAutor(my_str):
    my_str1 = re.sub("\['", "", my_str)
    my_str2 = re.sub("'\]", "", my_str1)
    my_str3 = re.sub("'", "", my_str2)
    return(my_str3)

In [3]:
def listdir_path(d):
    # Return full path of all files & directories in directory
    list_full_path = []
    for path in os.listdir(d):
        full_path = os.path.join(d, path)
        list_full_path.append(full_path)
    return list_full_path

In [4]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent=" ")

In [5]:
# That´s how the TEIs supposed to look like (following the GROBID "processHeader" batch command output)
from IPython.display import Image
img = 'template_tei.png'
Image(url=img)

## Extract Core Data

In [6]:
df_meta = pd.read_csv("metadata_15553.csv")
df_meta

,coreId,title,authors
0,11083759,Середовище проведення освітніх вебінарів «WIP ...,"['Богачков, Ю.М.']"
1,11083794,Комплексна підготовка дистанційних матеріалів,"['Богачков, Ю.М.', 'Царенко, В.О.']"
2,11083801,Метод аналізу елементарних доменів для покраще...,"['Богачков, Ю.М.', 'Ухань, П.С.', 'Полобюк, Т...."
3,11083807,Інструментальні засоби профорієнтації,"['Богачков, Ю.М.', 'Буров, О.Ю.', 'Бельська, Н..."
4,11083814,Експеримент «Дистанційне навчання школярів”: п...,"['Богачков, Ю.М.']"
...,...,...,...
15548,95313001,Підігрів тротуарів за допомогою теплових мереж,"['Семенюк, К.А.']"
15549,95313006,Шлакозоловидалення як спосіб підвищення енерго...,"['Бондар, Г.Ю.']"
15550,95313010,Підвищення ефективності роботи каміну,"['Півень, Д.В.']"
15551,95313012,Технологія поквартирного обліку теплової енергії,"['Горбачьова, А.О.']"


## Write TEI 

In [ ]:
os.chdir("/TEI_files") # Path where to store the final TEIs

In [113]:
for c in range(len(df_meta.coreId)):
    # first read in the template TEI. From there we will fill up the entries.
    tree = ET.parse("/template.tei") # Path to the "template.tei"
    template_root = tree.getroot()
    #################################
    titleStmt = template_root[0][0][0]
    analytic = template_root[0][0][2][0][0]

    # get meta data
    paper_meta = df_meta.loc[c]

    core_id = paper_meta.coreId
    title = paper_meta.title
    authors = paper_meta.authors

    ### Get autor for the PDF´s
    autor_pdf  = authors

    authors = removeAutor(authors).split(",")
    for j in range(len(authors)):
        authors[j] = ' '.join(authors[j].split()) ## remove excessive whitespace

    authors

    # construct tei
    title1 = SubElement(titleStmt, 'title', {"level":"a" , "type":"main"})
    title1.text = title

    for i in range(len(authors)):
        if i%2 == 0:
            author = SubElement(analytic , "author")
            persName = SubElement(author, "persName" , {"xmlns":"http://www.tei-c.org/ns/1.0"})
            surname  = SubElement(persName, "surname")
            surname.text = authors[i]
        else:
            if re.match('.\.',authors[i]) != None:
                author_forenames = authors[i].split(".")
            else: 
                author_forenames = authors[i].split(" ")
            forename  = SubElement(persName, "forename" , {"type": "first"})
            forename.text = author_forenames[0]
            try: 
                forename  = SubElement(persName, "forename" , {"type": "middle"})
                forename.text = author_forenames[1]
            except:
                pass

    # write tei
    file_name = "Core_ID_" + str(core_id) +".tei.xml"
    mydata = str(prettify(template_root)).replace("ns0:","").replace(":ns0","")
    myfile = open(file_name, "w",encoding="utf-8")
    myfile.write(mydata)
    myfile.close()